#### Ford-Fulkerson (FF) Alrogithm for Maximum Flow

We will first implement FF without scaling. We will use BFS traversal to find s-t paths.

In [57]:
# use BFS to find path from s to t in Gf
def find_st_path(adjacency_list, s, t):
    assert (s >= 0 and s < len(adjacency_list)), "s is out of range"
    # initialize all vertices as undiscovered except for s
    discovered = [False] * len(adjacency_list)
    discovered[s] = True
    parent = [None] * len(adjacency_list)
    # intiialize BFS queue
    Q = []
    Q.append(s)
    # initialize s-t path
    path = []
    # run BFS
    while len(Q)>0:
        # pop vertex from front of queue
        u = Q.pop(0)
        # find vertices adjacent to u and add them to the Q if they are undiscovered
        for (v,ce) in adjacency_list[u]:
            if not discovered[v]:
                discovered[v] = True
                parent[v] = (u, ce)
                # if v is t, then we have found the path
                if v == t:
                    # reconstruct path from s to t following parent pointers
                    while parent[t] != None:
                        # add edge to path
                        edge = (parent[t][0], t, parent[t][1])
                        path.append(edge)
                        t = parent[t][0]
                    return path
                # add to Q
                Q.append(v)
 
    # if we reach here, then there is no s-t path
    return path            


def construct_residual_graph(adjacency_list, flow, capacity):
    Gf_adjacency_list = {u:[] for u in adjacency_list.keys()}
    for u in range(len(adjacency_list)):
        for (v, ce) in adjacency_list[u]:
            # add forward edge if there's non-zero residual capacity
            residual_ce = capacity[(u, v)] - flow[(u, v)]
            if residual_ce > 0:
                Gf_adjacency_list[u].append((v, residual_ce))
            # add backward edge if there's non-zero flow 
            if flow[(u, v)] > 0:
                Gf_adjacency_list[v].append((u, flow[(u, v)]))

    return Gf_adjacency_list            


def FF_nosclaing(adjacency_list, verbose=False):
    # initialize the flow to 0
    flow = {}
    capacity = {}
    for u in adjacency_list.keys():
        for (v, ce) in adjacency_list[u]:
            flow[(u, v)] = 0
            capacity[(u, v)] = ce

    # construct residual graph
    Gf_adjacency_list = construct_residual_graph(adjacency_list, flow, capacity)
    if verbose:
        print("Initial residual graph:")
        print(Gf_adjacency_list)

    s = 0
    t = len(adjacency_list)-1 
    num_iterations = 0
    # run augmenting iterations
    while True:
        # find s-t path in Gf
        path = find_st_path(Gf_adjacency_list, s, t)
        # if there is no s-t path, then we are done
        if len(path) == 0:
            if verbose:
                flow_value = sum([flow[(s, v)] for (v, _) in adjacency_list[s]])
                print(f"\nMax flow found! |f| = {flow_value}, total iterations = {num_iterations}")
            return flow
        else:
            # find bottleneck capacity of path
            bottleneck_capacity = float('inf')
            for (u, v, ce) in path:
                if ce < bottleneck_capacity:
                    bottleneck_capacity = ce
            # augment flow along path
            for (u, v, ce) in path:
                if (u, v) in capacity:
                    # increase flow along forward edge (u, v)
                    flow[(u, v)] += bottleneck_capacity
                else:
                    # decrease flow along backward edge (v, u)
                    flow[(v, u)] -= bottleneck_capacity 
            # construct updated residual graph        
            Gf_adjacency_list = construct_residual_graph(adjacency_list, flow, capacity)     
            
            if verbose:
                print(f"\nFlow augmented along path: {path} by amount: {bottleneck_capacity}")
                print(f"New residual graph: \n {Gf_adjacency_list}")
                print(f"Augmented Flow: \n {flow}")   

            num_iterations += 1                   

Example flow network:

<img src="example_1.png" width="450" height="230">


In [58]:
# adjacenecy list of each node contains a tuple of (node, capacity), node 0 is the source and node 5 is the sink
adjacency_list = {0:[(1,2), (3,3)], 1:[(2,5), (4,3)], 2:[(5,2)], 3:[(2,1)], 4:[(5,4)], 5:[]}

In [59]:
# run Ford-Fulkerson algorithm
flow = FF_nosclaing(adjacency_list, verbose=True)

Initial residual graph:
{0: [(1, 2), (3, 3)], 1: [(2, 5), (4, 3)], 2: [(5, 2)], 3: [(2, 1)], 4: [(5, 4)], 5: []}

Flow augmented along path: [(2, 5, 2), (1, 2, 5), (0, 1, 2)] by amount: 2
New residual graph: 
 {0: [(3, 3)], 1: [(0, 2), (2, 3), (4, 3)], 2: [(1, 2)], 3: [(2, 1)], 4: [(5, 4)], 5: [(2, 2)]}
Augmented Flow: 
 {(0, 1): 2, (0, 3): 0, (1, 2): 2, (1, 4): 0, (2, 5): 2, (3, 2): 0, (4, 5): 0}

Flow augmented along path: [(4, 5, 4), (1, 4, 3), (2, 1, 2), (3, 2, 1), (0, 3, 3)] by amount: 1
New residual graph: 
 {0: [(3, 2)], 1: [(0, 2), (2, 4), (4, 2)], 2: [(1, 1), (3, 1)], 3: [(0, 1)], 4: [(1, 1), (5, 3)], 5: [(2, 2), (4, 1)]}
Augmented Flow: 
 {(0, 1): 2, (0, 3): 1, (1, 2): 1, (1, 4): 1, (2, 5): 2, (3, 2): 1, (4, 5): 1}

Max flow found! |f| = 3, total iterations = 2
